In [1]:
from BiblioMeter_GUI.Page_Classes import App_Test

app = App_Test()
app.mainloop()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LD259969\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LD259969\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LD259969\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Début du brouillon

In [ ]:
# AJOUTER AUTOMATIQUEMENT CETTE FONCTION A LA PROCEDURE DONE

import pandas as pd
from BiblioMeter_FUNCTS.BiblioMeterFonctions import add_authors_name_list

in_path = r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2022/BDD multi mensuelle/submit.xlsx'
out_path = r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2022/BDD multi mensuelle/submit.xlsx'

add_authors_name_list(in_path, out_path)

In [ ]:
import pandas as pd
in_path = r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2022/BDD multi mensuelle/submit.xlsx'
out_path = r'S:/130-LITEN/130.1-Direction/130.1.2-Direction Scientifique/130.1.2.1-Dossiers en cours/111- Ludovic Desmeuzes/BiblioMeter_Files/2022/BDD multi mensuelle/cleaned_submit.xlsx'

In [ ]:
from BiblioMeter_FUNCTS import clean_reorder_rename_submit
clean_reorder_rename_submit(in_path, out_path)
# A FINIR APRES IMPACT FACTOR

In [ ]:
# Notes pour écrire dans un document texte
path = r'S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\111- Ludovic Desmeuzes\BiblioMeter_Files\Listing RH\MAJ.txt'
f = open(path,'r')
print(f.readline())
f.close()

f = open(path,'w')
nouvelle_date = f'{date.today().month}/{date.today().year}'
f.writelines(nouvelle_date)
f.close()

In [ ]:
def maj_listing_RH():

    def date_compare(list_of_dates, comparator):
        '''
        Args:
            list_of_dates (list of strings): a list of string of dates in format mmyyyy, mm being the number of the month and yyyy being the number of the year.
            comparator (string): a date in string format mmyyyy, mm being the number of the month and yyyy being the number of the year.

        Returns:
            L (list of strings): a list of the dates older than the comparator date.

        '''
        L = []
        for i in list_of_dates:
            if i[2:6] > comparator[2:6]:
                # Alors ok pas besoin de vérifier
                L.append(i)
            elif i[2:6] == comparator[2:6]:
                if i[0:2] > comparator[0:2]:
                    # Okay si le mois est supérieur
                    L.append(i)
        return L

    def concat_df_with_multidf_and_drop_dup(multi_df, df = pd.DataFrame()):
        '''
        Args:
            multi_df (DataFrame):
            df (DataFrame):
        Returns:
            df (DataFrame):
        '''
        for i in range(len(multi_df)):
            clef = list(df_month.keys())[i]
            df = df.append(multi_df[clef])
        df.drop_duplicates(subset=['Matricule'], keep='first', inplace=True, ignore_index=False)
        return df

    def different_years(list_of_dates):
        '''
        Args:
            list_of_dates (list of strings): a list of string of dates in format mmyyyy, mm being the number of the month and yyyy being the number of the year.
        Returns:
            L (list of strings):
        '''
        L = []
        for i in list_of_dates:
            L.append(i[2:6])
        return list(set(L))

    def dates_of_the_given_year(given_year, list_of_dates):
        '''
        Args:
            given_year (string):
            list_of_dates (list of strings): a list of string of dates in format mmyyyy, mm being the number of the month and yyyy being the number of the year.
        Returns:
            L (list of strings):
        '''
        L = []
        for i in list_of_dates:
            if i[2:6] == given_year:
                L.append(i)
        return L

    # Mise à jour fichier RH

    import pandas as pd 
    import numpy as np 
    from openpyxl import load_workbook 

    path_year = r"S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\111- Ludovic Desmeuzes\BiblioMeter_Files\Listing RH\All_effectifs.xlsx" 
    path_month = r"S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\111- Ludovic Desmeuzes\BiblioMeter_Files\Listing RH\Effectifs_2010_2022.xlsx" 
    path_maj = r'S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\111- Ludovic Desmeuzes\BiblioMeter_Files\Listing RH\MAJ.txt'

    # Récupérer la date de la dernière MAJ
    f = open(path_maj,'r')
    last_maj = f.readline()
    f.close()

    # Récupérer les pages dont les dates sont antérieures à last_maj
    xls = pd.ExcelFile(path_month, engine = 'openpyxl')
    sheets = xls.sheet_names
    excel_sheets = date_compare(sheets, last_maj)
    print(f"Toutes les pages à maj sont {excel_sheets}")

    # Récupérer les différentes années et boucler dessus
    diff_years = different_years(excel_sheets)
    print(f"Les années différentes sur ces pages sont {diff_years}")

    # Ouverture du workbook et writer
    book = load_workbook(path_year)
    writer = pd.ExcelWriter(path_year, engine = 'openpyxl', mode = 'a', if_sheet_exists = 'replace') 
    # writer.book = book

    for year in diff_years:
        # Les dates des pages à rajouter pour l'année en question
        month_pages = dates_of_the_given_year(year, excel_sheets)
        print(f"Pour l'année {year}, les mois sont {month_pages}")

        # La multi df à récupérer de l'année en question pour mettre à jour le fichier effectif de l'année
        df_month = pd.read_excel(path_month, sheet_name = month_pages)

        # Vérifier l'excistence de la page de l'année de path_year, et si elle existe la récupérer, sinon la créer.
        try:
            print('Test du try')
            df_year = pd.read_excel(path_year, sheet_name = year, engine = 'openpyxl')

            print(f"La page existe ! Tout est bon, on peut continuer")
            df_maj = concat_df_with_multidf_and_drop_dup(df_month, df_year)
            #book.remove(year)
            df_maj.to_excel(writer, sheet_name = year)

            print('Fin du try')

        except:
            print(f"La page n'existe pas ! Il faut donc la créer lors de l'ajout de la DataFrame")
            df_maj = concat_df_with_multidf_and_drop_dup(df_month)
            df_maj.to_excel(writer, sheet_name = year)

            print('Fin du except')

    writer.save()
    writer.close()

    print('Terminé, vous pouvez ouvrir le fichier excel')

In [ ]:
def ajout_IF(in_path, out_path, IF_path, year):
    
    ''' 

    Args:
        in_path (path): path (including name of the file if year != None) leading to the working excel file. 
        out_path (path): path (including name of the file) leading to where the file will be saved after going through its treatment.
        IF_path (path): path (including name of the file) leading to the impact factor excel file.
        year (int):
    
    Returns:
    
    Notes:
    
    '''
    
    print(f"in_path = {in_path}")
    
    # Standard imports
    from pathlib import Path

    # Local imports
    from BiblioMeter_FUNCTS.BiblioMeterGlobalsVariables import COL_NAMES_BONUS
    from BiblioAnalysis_Utils.BiblioSpecificGlobals import COL_NAMES
    from BiblioMeter_FUNCTS.BiblioMeterGlobalsVariables import FILL_EMPTY_KEY_WORD

    # 3rd party imports
    import pandas as pd

    # Useful alias
    ISSN_alias = COL_NAMES['articles'][10]
    EISSN_alias = COL_NAMES_BONUS['EISSN']
    IF_alias = COL_NAMES_BONUS['If clarivate']
    IF_cours_alias = COL_NAMES_BONUS['IF en cours']
    IF_publi_alias = COL_NAMES_BONUS['IF année publi']
    
    df_submit = pd.read_excel(in_path) # Ma DataFrame
    
    if year == None:
        df_IF = pd.read_excel(IF_path, sheet_name = None)
        
        # using dictionary to convert type of  Year column
        convert_dict = {'Year': str}
        df_submit = df_submit.astype(convert_dict)

        list_annee = list(df_submit['Year'].unique())
        try :
            list_annee.remove('unknown')
        except ValueError:
            pass
        
        # Maintenant qu'on a les années, on boucle dessus pour appliquer un filtre
        df_submit_bis = pd.DataFrame()
        for annee in list_annee:
            print(annee)
            df_inter = df_submit[df_submit['Year'] == annee]
            
            try:
                dict1 = dict(zip(df_IF[str(int(annee) + 1)][ISSN_alias], df_IF[str(int(annee) + 1)][IF_alias]))
                
                s = df_inter[ISSN_alias] # Je récup la colonne clef de ma DF en Series
                
                r = s.map(dict1) # Je map la Series avec mon dictionnaire
                                
                df_inter[Impact_Factor_alias] = r # Je rajoute la colonne à ma DataFrame
                
                # Appliquer nan --> unknow to df
                df_inter[Impact_Factor_alias] = df_inter[Impact_Factor_alias].fillna(FILL_EMPTY_KEY_WORD)

                df_submit_bis = df_submit_bis.append(df_inter)
                
            except KeyError:
                dict1 = dict(zip(df_IF[str(annee)][ISSN_alias], df_IF[str(annee)][IF_alias]))
                
                s = df_inter[ISSN_alias] # Je récup la colonne clef de ma DF en Series
                
                r = s.map(dict1) # Je map la Series avec mon dictionnaire
                
                df_inter[Impact_Factor_alias] = r # Je rajoute la colonne à ma DataFrame
                
                # Appliquer nan --> unknow to df
                df_inter[Impact_Factor_alias] = df_inter[Impact_Factor_alias].fillna(FILL_EMPTY_KEY_WORD)

                df_submit_bis = df_submit_bis.append(df_inter)
            
        df_submit_bis.to_excel(out_path, index = False)
                
    
    else: # Mode de fonctionnement par année
        # Check if the year + 1 is available
        try:
            df_IF = pd.read_excel(IF_path, sheet_name = str(year + 1))
            # Ca fonctionne, alors on ajoute la colonne IF de l'année en cours
            print(f"Les IF sortis en {year +1} sont utilisés pour créer la colonne f{IF_cours_alias}")

            dict1 = dict(zip(df_IF[ISSN_alias], df_IF[IF_alias])) # Mon dictionnaire construit à partir de mon fichier excel
            s = df_submit[ISSN_alias] # Je récup la colonne clef de ma DF en Series
            r = s.map(dict1) # Je map la Series avec mon dictionnaire
            df_submit[IF_cours_alias] = r # Je rajoute la colonne à ma DataFrame

            # Appliquer nan --> unknow to df TO DO
            df_submit[IF_cours_alias] = df_submit[IF_cours_alias].fillna(FILL_EMPTY_KEY_WORD)
            
            # Et on fait pareil avec les IF de l'année de publication
            print(f"Les IF sortis en {year} sont utilisés pour créer la colonne f{IF_publi_alias}")

            dict1 = dict(zip(df_IF[ISSN_alias], df_IF[IF_alias])) # Mon dictionnaire construit à partir de mon fichier excel
            s = df_submit[ISSN_alias] # Je récup la colonne clef de ma DF en Series
            r = s.map(dict1) # Je map la Series avec mon dictionnaire
            df_submit[IF_publi_alias] = r # Je rajoute la colonne à ma DataFrame

            # Appliquer nan --> unknow to df
            df_submit[IF_publi_alias] = df_submit[IF_publi_alias].fillna(FILL_EMPTY_KEY_WORD)
            
            df_submit.to_excel(out_path, index = False)
                
        except ValueError:
            # Check if the year is available, if not, terminate function
            try:
                df_IF = pd.read_excel(IF_path, sheet_name = str(year))
                print(f"Les IF sortis en {year} sont utilisés")
                
                dict1 = dict(zip(df_IF[ISSN_alias], df_IF[IF_alias])) # Mon dictionnaire construit à partir de mon fichier excel
                
                s = df_submit[ISSN_alias] # Je récup la colonne clef de ma DF en Series
                
                r = s.map(dict1) # Je map la Series avec mon dictionnaire
                
                df_submit[Impact_Factor_alias] = r # Je rajoute la colonne à ma DataFrame
                
                # Appliquer nan --> unknow to df
                df_submit[Impact_Factor_alias] = df_submit[Impact_Factor_alias].fillna(FILL_EMPTY_KEY_WORD)

                df_submit.to_excel(out_path, index = False)
                
            except ValueError:
                print('Mettre à jour le fichier Impact Factor')

In [ ]:
import pandas as pd
import os
from pathlib import Path
in_path = Path(r"S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\110-Alternants\2021-22 Ludovic Desmeuzes\BiblioMeter_Files\BDD multi annuelle") / Path(os.listdir(r"S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\110-Alternants\2021-22 Ludovic Desmeuzes\BiblioMeter_Files\BDD multi annuelle")[-2])
IF_path = Path(r"S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\110-Alternants\2021-22 Ludovic Desmeuzes\BiblioMeter_Files\Impact Factor\IF all years.xlsx")

In [ ]:
in_path

In [ ]:
from BiblioMeter_FUNCTS.BiblioMeterFonctions import ajout_IF
ajout_IF(in_path, in_path, IF_path, None)

In [ ]:
try:
    df_IF = pd.read_excel(IF_path, sheet_name = '2023')
except ValueError:
    try:
        df_IF = pd.read_excel(IF_path, sheet_name = '2022')
        print('Tout a fonctionné')
    except ValueError:
        print('Mettre à jour le fichier Impact Factor')

In [ ]:
'2023' in df_IF.keys()

In [ ]:
import pandas as pd
df_IF = pd.read_excel(IF_path)

In [ ]:
import os
from pathlib import Path

In [ ]:
in_path = Path(r"S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\110-Alternants\2021-22 Ludovic Desmeuzes\BiblioMeter_Files\BDD multi annuelle")

In [ ]:
os.listdir(in_path)

In [ ]:
list(df_IF.keys())[-1]

In [ ]:
from pathlib import Path
import pandas as pd
IF_path = Path(r"S:\130-LITEN\130.1-Direction\130.1.2-Direction Scientifique\130.1.2.1-Dossiers en cours\110-Alternants\2021-22 Ludovic Desmeuzes\BiblioMeter_Files\Impact Factor\IF all years.xlsx")

df_IF_2021 = pd.read_excel(IF_path, sheet_name = "2021")
df_IF_2020 = pd.read_excel(IF_path, sheet_name = "2020")

dict1 = dict(zip(df_IF_2021['Journal Majuscule'], df_IF_2021['ISSN'])) # Création dictionnaire avec les noms des journaux de la fiche 2020 et des ISSN dispo de l’année 2022

s = df_IF_2020['Journal Majuscule'] # Mettre en type série la colonne avec les noms des journaux (car le map ne fonctionne qu’avec les séries)

r = s.map(dict1) # Effectuer le map avec la série et le dictionnaire

df_IF_2020['ISSN'] = r # Rajouter le map (r) à la DF

df_IF_2020.to_excel(IF_path, sheet_name = "2020 new", header = True) # Envoyer sur le fichier excel 

In [ ]:
print(df_IF['JCR Abbreviated Title'][1299])

In [ ]:
a = 'salut cOmmEnt ça Va ?'
b = a.title()
b

In [ ]:
df_IF

In [5]:
import tkinter as tk

def _place_after(w,cible,dx=5,dy=0):
    cible_info = cible.place_info()
    x = dx + int(cible_info['x']) + cible.winfo_reqwidth()
    y = int(cible_info['y']) + cible.winfo_reqheight()/2 + dy
    w.place(x = x, y = y, anchor = 'w')
    
def _encadre_label_button(fond, label, button, dx = 10, dy = 10):
    
    fond.delete('all')
    label_info = label.place_info()
    button_info = button.place_info()

    x1 = int(button_info['x']) - dx
    y1 = int(button_info['y']) - dy
    x2 = int(button_info['x']) + button.winfo_reqwidth() + label.winfo_reqwidth() + dx
    y2 = int(label_info['y']) + max(button.winfo_reqheight()/2, label.winfo_reqheight()) + dx

    rectangle = fond.create_rectangle(x1, y1, x2, y2, outline = "red", width = 2)
    fond.place(x = 0, y = 0)

class App:
    def __init__(self):
        self.root = tk.Tk()
        x_root = 500
        y_root = 500
        self.root.geometry(f"{x_root}x{y_root}")
        self.fond = tk.Canvas(self.root, width =  x_root, height =  y_root)
        
        
        # Vous pouvez vous amuser à déplacer le bouton, et à jouer sur sa taille (limite 500 x 500)
        self.b = tk.Button(self.root, height = 3)
        self.b.place(x=220, y=85)
        
        self.l = tk.Label(self.root, text='ya plus dsauc\'')
        tk.Scale(self.root, from_=0, to=20, command=self.actu,
                 orient='horizontal').place(x=17,y=79)
        
        self.root.mainloop()
        
    def actu(self, arg):
        self.b['text'] = 'gretchipdertubachagtirengo'[:int(arg)]
        self.b['width'] = len('gretchipdertubachagtirengo'[:int(arg)])
        _place_after(self.l, self.b, dy=2)
        _encadre_label_button(self.fond, self.l, self.b)


if __name__ == '__main__':
    App()